In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install numdifftools
%pip install seaborn

In [2]:
dataset_size = 1000
neighborhood_size=100

# Create synthetic classification dataset

# Make moons

In [14]:
X,y = sklearn.datasets.make_moons(n_samples=dataset_size,  noise=0.25, random_state=0)#factor=0.5
Xdf = pd.DataFrame(X, columns=['x1','x2'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X), y, test_size=0.33, random_state=42)
X_train.columns = ['x1','x2']
X_test.columns = ['x1','x2']

In [ ]:
X_train.plot(kind='scatter',x='x1', y='x2',style='.', c=y_train, colormap='viridis')

In [ ]:
#xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
from sklearn.svm import SVC
svc = SVC(kernel='rbf',probability=True)
svc.fit(X_train, y_train)
svc_preds = svc.predict(X_test)
print(accuracy_score(y_test, svc_preds))

print(classification_report(y_test, svc_preds))

In [18]:
from sklearn.linear_model import LogisticRegression
xclf = RandomForestClassifier()
xclf.fit(X_train, y_train)
lr_preds = xclf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, lr_preds))

print(classification_report(y_test, lr_preds))

In [ ]:
fig, ax = plt.subplots()
point = [1.0,0.0]
ax.plot(point[0], point[1], 'or', markersize=8)
X_test.plot(kind='scatter',x='x1', y='x2', c=y_test, colormap='viridis', ax=ax)

In [21]:
X_test_appended = pd.concat((X_test,pd.DataFrame([point], columns=['x1','x2'])))
y_test_appended = np.concatenate([y_test,[1]])

## Decision Boundaries

In [ ]:

h = .02  # step size in the mesh
# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))


# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
ZZ = xclf.predict(pd.DataFrame(np.c_[xx.ravel(), yy.ravel()], columns=['x1','x2']))


# Put the result into a color plot
Z = Z.reshape(xx.shape)
fig,ax= plt.subplots(1,2, figsize=(12,3))
ax[0].contour(xx, yy, Z, cmap=plt.cm.Paired)
Xdf.plot(kind='scatter',x='x1', y='x2',style='.', c=y, colormap='viridis',ax=ax[0])

ZZ = ZZ.reshape(xx.shape)
ax[1].contour(xx, yy, ZZ, cmap=plt.cm.Paired)
X_train.plot(kind='scatter',x='x1', y='x2',style='.', c=y_train, colormap='viridis',ax=ax[1])
ax[0].set(title='SVM decision boundary')
ax[1].set(title='XGBoost decision boundary')
fig.savefig('dataset.png')

## Confidence plots

In [ ]:
import matplotlib.pylab as plt
from matplotlib import colors,cm

fig, ax = plt.subplots(1, 2, figsize=(12,3))

y_test_proba = svc.predict_proba(X_test_appended)
confidence_svc =   pd.DataFrame(MinMaxScaler().fit_transform(pd.DataFrame({'SVC_0':y_test_proba[:,0], 'SVC_1':y_test_proba[:,1]})))

y_test_proba_xg = xclf.predict_proba(X_test_appended)
confidence_xgb =   pd.DataFrame(MinMaxScaler().fit_transform(pd.DataFrame({'XGB_0':y_test_proba_xg[:,0],'XGB_1':y_test_proba_xg[:,1]})))

X_test_1 = X_test_appended.iloc[y_test_appended==1]
X_test_2 = X_test_appended.iloc[y_test_appended==0]
X_test_1.columns=['x1','x2']

cmapp = cm.viridis


c=0
cmap = colors.LinearSegmentedColormap.from_list(
        'incr_alpha', [(0, (*tuple(cmapp.colors[c]),0)), (1, cmapp.colors[c])])
ax[0].scatter(X_test_1['x1'], X_test_1['x2'], c=confidence_svc.iloc[y_test_appended==1,1].values, cmap=cmap)#, ec=None, s=10**2)


c=int(255*.5)
cmap = colors.LinearSegmentedColormap.from_list(
        'incr_alpha', [(0, (*tuple(cmapp.colors[c]),0)), (1, cmapp.colors[c])])
ax[0].scatter(X_test_2['x1'], X_test_2['x2'], c=confidence_svc.iloc[y_test_appended==0,0].values, cmap=cmap)#, ec=None, s=10**2)
ax[0].set(title='SVM uncertainty in predicions')


c=0

cmap = colors.LinearSegmentedColormap.from_list(
        'incr_alpha', [(0, (*tuple(cmapp.colors[c]),0)), (1, cmapp.colors[c])])
ax[1].scatter(X_test_1['x1'], X_test_1['x2'], c=confidence_xgb.iloc[y_test_appended==1,1].values, cmap=cmap)#, ec=None, s=10**2)


c=int(255*.5)
cmap = colors.LinearSegmentedColormap.from_list(
        'incr_alpha', [(0, (*tuple(cmapp.colors[c]),0)), (1, cmapp.colors[c])])
ax[1].scatter(X_test_2['x1'], X_test_2['x2'], c=confidence_xgb.iloc[y_test_appended==0,0].values, cmap=cmap)#, ec=None, s=10**2)
ax[1].set(title='XGB uncertainty in predictions')

ax[0].plot(point[0], point[1], 'or', markersize=8)
ax[1].plot(point[0], point[1], 'or', markersize=8)


fig.savefig('uncertainty-point.png')

h = .02  # step size in the mesh
# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))


# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, m_max]x[y_min, y_max].
Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
ZZ = xclf.predict(pd.DataFrame(np.c_[xx.ravel(), yy.ravel()], columns=['x1','x2']))


# Put the result into a color plot
Z = Z.reshape(xx.shape)
fig,ax= plt.subplots(1,2, figsize=(12,3))
ax[0].contour(xx, yy, Z, cmap=plt.cm.Paired)
ax[0].scatter(X_test['x1'], X_test['x2'], c=y_test, cmap='viridis')

ZZ = ZZ.reshape(xx.shape)
ax[1].contour(xx, yy, ZZ, cmap=plt.cm.Paired)
ax[1].scatter(X_test['x1'], X_test['x2'], c=y_test, cmap='viridis')
ax[0].set(title='SVM decision boundary')
ax[1].set(title='XGBoost decision boundary')

ax[0].plot(point[0], point[1], 'or', markersize=8)
ax[1].plot(point[0], point[1], 'or', markersize=8)
fig.savefig('decbound-point.png')
plt.show()

## Lime explanations

In [ ]:
point

In [16]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=list(X_train.columns), class_names=['0','1'], discretize_continuous=True)

In [ ]:
idx = len(X_test_appended)-1
exp = explainer.explain_instance(X_test_appended.iloc[idx,:], svc.predict_proba, num_features=2, top_labels=1)

In [ ]:
exp.show_in_notebook(show_table=True, show_all=False)

## Shap explanations

In [ ]:
shap.initjs()

In [ ]:
point

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# use Kernel SHAP to explain test set predictions
explainer = shap.KernelExplainer(svc.predict_proba, X_train, link="logit")
shap_values = explainer.shap_values(X_test_appended, nsamples=100)

# plot the SHAP values for the Setosa output of the first instance
idx = len(X_test_appended)-1
shap.force_plot(explainer.expected_value[0], shap_values[0][idx,:], X_test_appended.iloc[idx,:], link="logit")

## Anchor explanations

In [21]:
explainer = anchor_tabular.AnchorTabularExplainer(
    ['0','1'],
    feature_names = X_train.columns,
    train_data = X_train.values,
    categorical_names={})

In [ ]:
np.random.seed(1)
print('Prediction: ', explainer.class_names[svc.predict(X_test_appended.iloc[idx].values.reshape(1,-1))[0]])
exp = explainer.explain_instance(X_test_appended.iloc[idx].values, svc.predict, threshold=0.95)

In [ ]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

## LUX explanations

In [24]:
lux = LUX(predict_proba = svc.predict_proba, neighborhood_size=100,max_depth=2,  node_size_limit = 1, grow_confidence_threshold = 0 )
lux.fit(X_train, y_train, instance_to_explain=point,class_names=[0,1])

In [26]:
preds =[int(v) for v in lux.predict(X_train)]
sum(preds==y_train)/len(y_train)

### Single rule justification

In [ ]:
lux.justify(np.array([point]))

### Counterfactual explanation

In [ ]:
lux.counterfactual(np.array([point]), X_train, counterfactual_representative='nearest', topn=1)

### Whole tree justification

In [ ]:
print(lux.to_HMR())

In [1]:
import sys
#sys.path.insert(0, r'path-to-original-src-folder')
sys.path.insert(0, r'path-to-modified-src-folder')

# Iris example

In [ ]:
from lux.lux import LUX
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import pandas as pd
from itertools import product
# import some data to play with
iris = datasets.load_iris()
features = ['sepal_length','sepal_width','petal_length','petal_width']
target = 'class'

#create daatframe with columns names as strings (LUX accepts only DataFrames withj string columns names)
df_iris = pd.DataFrame(iris.data,columns=features)
df_iris[target] = iris.target

#train classifier
train, test = train_test_split(df_iris)
clf = svm.SVC(probability=True)
clf.fit(train[features],train[target])
clf.score(test[features],test[target])

## ORIGINAL CODE

In [ ]:
iris_instance = train[features].sample(1).values
iris_instance

parameters = ['inverse_sampling', 'density_sampling', 'radius_sampling', 'oversampling']
combinations = list(product([True, False], repeat=len(parameters)))
for combination in combinations:
    params = dict(zip(parameters, combination))
    print(f"Testing parameters: {params}")

    lux = LUX(
        predict_proba=clf.predict_proba,
        neighborhood_size=20,
        max_depth=2,
        node_size_limit=1,
        grow_confidence_threshold=0
    )

    lux.fit(
        train[features],
        train[target],
        instance_to_explain=iris_instance,
        class_names=[0, 1, 2],
        **params
    )

    justification = lux.justify(np.array(iris_instance))
    print(f"Justification: {justification}")

    cf = lux.counterfactual(
        np.array(iris_instance),
        train[features],
        counterfactual_representative='nearest',
        topn=1
    )[0]
    print(f"Counterfactual for {iris_instance} to change from class {lux.predict(np.array(iris_instance))[0]} to class {cf['prediction']}: \n{cf['counterfactual']}")

    print(lux.to_HMR())
    print("-" * 50)

In [ ]:
import graphviz
from graphviz import Source
from IPython.display import SVG, Image
i2edf = pd.DataFrame(iris_instance, columns=features) 
i2edf[target] =clf.predict(i2edf.values.reshape(1,-1))[0] 
cfdf = pd.DataFrame(cf['counterfactual']).T
cfdf[target] = clf.predict(cfdf.values.reshape(1,-1))[0]
lux.uid3.tree.save_dot('tree.dot',fmt='.2f',visual=True, background_data=train, instance2explain=i2edf, counterfactual=cfdf)
gvz=graphviz.Source.from_file('tree.dot')
!dot -Tpng tree.dot > tree.png
Image('tree.png')

## MODIFIED CODE

In [1]:
import sys
#sys.path.insert(0, r'path-to-original-src-folder')
sys.path.insert(0, r'path-to-modified-src-folder')

In [ ]:
for combination in combinations:
    params = dict(zip(parameters, combination))
    print(f"Testing parameters: {params}")

    lux = LUX(
        predict_proba=clf.predict_proba,
        neighborhood_size=20,
        max_depth=2,
        node_size_limit=1,
        grow_confidence_threshold=0
    )

    lux.fit(
        train[features],
        train[target],
        instance_to_explain=iris_instance,
        class_names=[0, 1, 2],
        **params
    )

    justification = lux.justify(np.array(iris_instance))
    print(f"Justification: {justification}")

    cf = lux.counterfactual(
        np.array(iris_instance),
        train[features],
        counterfactual_representative='nearest',
        topn=1
    )[0]
    print(f"Counterfactual for {iris_instance} to change from class {lux.predict(np.array(iris_instance))[0]} to class {cf['prediction']}: \n{cf['counterfactual']}")

    print(lux.to_HMR())
    print("-" * 50)

## TUANDROMD DATASET ON UNMODIFIED LUX

In [1]:
import sys
#sys.path.insert(0, r'path-to-original-src-folder')
sys.path.insert(0, r'path-to-modified-src-folder')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from lux.lux import LUX
import numpy as np
from itertools import product
from sklearn.preprocessing import LabelEncoder

file_path = "TUANDROMD.csv"
df = pd.read_csv(file_path)

features = df.columns[:241]
target = 'Label'

encoder = LabelEncoder()
df[target] = encoder.fit_transform(df[target])

train, test = train_test_split(df)
clf = svm.SVC(probability=True)
clf.fit(train[features], train[target])
print(f"Dokładność modelu: {clf.score(test[features], test[target])}")

Dokładność modelu: 0.985663082437276


In [3]:
instance_to_explain = train[features].sample(1).values

parameters = ['inverse_sampling', 'density_sampling', 'radius_sampling', 'oversampling']
combinations = list(product([True, False], repeat=len(parameters)))
for combination in combinations:
    params = dict(zip(parameters, combination))
    print(f"Testowanie parametrów: {params}")

    lux = LUX(
        predict_proba=clf.predict_proba,
        neighborhood_size=20,
        max_depth=2,
        node_size_limit=1,
        grow_confidence_threshold=0
    )

    lux.fit(
        train[features],
        train[target],
        instance_to_explain=instance_to_explain,
        class_names=[0, 1],
        **params
    )

    with open(r"result1.txt", "a") as file:
        justification = lux.justify(np.array(instance_to_explain))
        file.write(f"Uzasadnienie: {justification}\n")

        cf = lux.counterfactual(
            np.array(instance_to_explain),
            train[features],
            counterfactual_representative='nearest',
            topn=1
        )[0]
        file.write(f"Przeciwprzykład dla {instance_to_explain}: {cf['counterfactual']}\n")
        file.write("-" * 50 + "\n")

Testowanie parametrów: {'inverse_sampling': True, 'density_sampling': True, 'radius_sampling': True, 'oversampling': True}


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_optics.py:1086: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\ja

Time:  32.901002407073975


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Testowanie parametrów: {'inverse_sampling': True, 'density_sampling': True, 'radius_sampling': True, 'oversampling': False}


...

c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Testowanie parametrów: {'inverse_sampling': False, 'density_sampling': False, 'radius_sampling': False, 'oversampling': False}


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Time:  0.017000675201416016


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


## TUANDROMD DATASET ON MODIFIED LUX

In [4]:
import sys
#sys.path.insert(0, r'path-to-original-src-folder')
sys.path.insert(0, r'path-to-modified-src-folder')

In [5]:
for combination in combinations:
    params = dict(zip(parameters, combination))
    print(f"Testowanie parametrów: {params}")

    lux = LUX(
        predict_proba=clf.predict_proba,
        neighborhood_size=20,
        max_depth=2,
        node_size_limit=1,
        grow_confidence_threshold=0
    )

    lux.fit(
        train[features],
        train[target],
        instance_to_explain=instance_to_explain,
        class_names=[0, 1],
        **params
    )

    with open(r"result2.txt", "a") as file:
        justification = lux.justify(np.array(instance_to_explain))
        file.write(f"Uzasadnienie: {justification}\n")

        cf = lux.counterfactual(
            np.array(instance_to_explain),
            train[features],
            counterfactual_representative='nearest',
            topn=1
        )[0]
        file.write(f"Przeciwprzykład dla {instance_to_explain}: {cf['counterfactual']}\n")
        file.write("-" * 50 + "\n")

Testowanie parametrów: {'inverse_sampling': True, 'density_sampling': True, 'radius_sampling': True, 'oversampling': True}


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_optics.py:1086: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
c:\Users\ja

Time:  24.377048015594482


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Testowanie parametrów: {'inverse_sampling': True, 'density_sampling': True, 'radius_sampling': True, 'oversampling': False}


...

c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Testowanie parametrów: {'inverse_sampling': False, 'density_sampling': False, 'radius_sampling': False, 'oversampling': False}


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Time:  0.029004335403442383


c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
c:\Users\jasie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
